In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [23]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [16]:
payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [17]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [18]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [19]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [14]:
@my_decorator
def inference(thread, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {thread}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], thread
    
        

In [22]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    print(len(prompts))
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompt}) for srno,prompt in enumerate(prompts, start=0)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

100
loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-0f9c497006ba4347b001fbe06c34d823","object":"text_completion","created":520855,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 4.736085 seconds 


Prompt 6==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-53043579d88149c1871b004458683e2d","object":"text_completion","created":520856,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt 2==> Problem: Write a function in python to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime. 
 {"id":"cmpl-f97e589060094feca3e179fb7a7a0107","object":"text_completion","created":520855,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef is_prime(num):\n    if num <= 1:\n        return False\n    elif num <= 3:\n        return True\n    elif num % 2 == 0 or num % 3 == 0:\n        return False\n    i = 5\n    while i * i <= num:\n        if num % i == 0 or num % (i + 2) == 0:\n            return False\n        i += 6\n    return True\n\n# Test the function\nprint(is_prime(2))  # True\nprint(is_prime(4))  # False\nprint(is_prime(17))  # True\nprint(is_prime(25))  # False\nprint(is_prime(1))  # False\nprint(is_prime(0))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":37,"total_tokens":217,

Execution time: 16.690140 seconds 



Execution time: 16.829818 seconds 


Execution time: 16.923620 seconds 


Prompt 23==> Problem: Write a function in c++ to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-a37bafab8e4b4b7eaffacc015c164c78","object":"text_completion","created":520856,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\n#include <string>\n\nbool isPalindrome(std::string s) {\n    int start = 0;\n    int end = s.length() - 1;\n\n    while (start < end) {\n        if (s[start] != s[end]) {\n            return false;\n        }\n        start++;\n        end--;\n    }\n    return true;\n}\n\nint main() {\n    std::string s;\n    std::cout << \"Enter a string: \";\n    std::cin >> s;\n\n    if (isPalindrome(s)) {\n        std::cout << \"The string is a palindrome.

Prompt 35==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-0ae6e88996d442068d2ae368f127b744","object":"text_completion","created":520864,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nFibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1. That is,\n\nFibonacci(0) = 0,\nFibonacci(1) = 1,\nFibonacci(n) = Fibonacci(n - 1) + Fibonacci(n - 2) for n > 1.\n\nHere is a simple implementation of the function in JavaScript:\n\n```javascript\nfunction fibonacci(n) {\n    if (n <= 1) {\n        return n;\n    } else {\n        return fibonacci(n - 1) + fibonacci(n - 2);\n    }\n}\n```\n\nThis function uses recursion to calculate the nth Fibonacci number. However, this","logprobs":null,"finish_reason":"length"}],

Prompt 39==> Problem: Write a function in javascript to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix. 
 {"id":"cmpl-3998d59095fe441d96f66012fca98758","object":"text_completion","created":520867,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nExample:\nInput: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\nOutput: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]\n\nHere is a JavaScript function that transposes a given matrix:\n\n```javascript\nfunction transposeMatrix(matrix) {\n    return matrix[0].map((col, i) => matrix.map(row => row[i]));\n}\n\nconsole.log(transposeMatrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]]));\n```\n\nThis function works by using the `map` function to create a new array where each element is a new array that contains the elements at the corresponding index of each input array. The result is a","logprobs":null,"finish_reason":"length

Execution time: 16.602921 seconds 


Prompt 63==> Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-eab3cecd289b430e824164f9aefab2b8","object":"text_completion","created":520874,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu using CSS and JavaScript.\n\nHTML:\n```html\n<button id=\"dropdownBtn\">Dropdown</button>\n<div id=\"dropdownMenu\">\n  <a href=\"#\">Link 1</a>\n  <a href=\"#\">Link 2</a>\n  <a href=\"#\">Link 3</a>\n</div>\n```\n\nCSS:\n```css\n#dropdownMenu {\n  display: none;\n  position: absolute;\n  background-color: #f9f9f9;\n  min-width: 160px;\n  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);\n  z-index: 1;\n}\n","logprobs":null,"finish_reason":"lengt

Prompt 70==> Problem: Implement an autocomplete feature in react for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions). 
 {"id":"cmpl-43c22e840e214562bae60fcd8a4e2c0a","object":"text_completion","created":520880,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can implement an autocomplete feature in React using a controlled component and the useState hook.\n\n```jsx\nimport React, { useState } from 'react';\n\nconst Autocomplete = () => {\n  const [inputValue, setInputValue] = useState('');\n  const [suggestions, setSuggestions] = useState([]);\n\n  const handleInputChange = (event) => {\n    const userInput = event.target.value;\n\n    // Filter suggestions based on user input\n    const filteredSuggestions = suggestions.filter(\n      (suggestion) =>\n        suggestion.toLowerCase().i

Prompt 77==> Problem: Write a function in react that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document). 
 {"id":"cmpl-558ad5116b2e4c969db7123f510c00c2","object":"text_completion","created":520886,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create a function in React that creates an HTML element and appends it to the document:\n\n```jsx\nimport React from 'react';\n\nclass App extends React.Component {\n  createElement(type, text) {\n    const element = document.createElement(type);\n    element.textContent = text;\n    document.body.appendChild(element);\n  }\n\n  render() {\n    return (\n      <div>\n        <button onClick={() => this.createElement('div', 'Hello, World!')}>\n          Create Div\n        </button>\n        <button onCli

Prompt 93==> Problem: Create an animated dropdown menu in jQuery that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-f7b6932654af4639b0436ed1e8160119","object":"text_completion","created":520891,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can create an animated dropdown menu using jQuery:\n\nHTML:\n```html\n<button id=\"dropdown-button\">Click me</button>\n<div id=\"dropdown-menu\" style=\"display: none;\">\n  <ul>\n    <li>Option 1</li>\n    <li>Option 2</li>\n    <li>Option 3</li>\n  </ul>\n</div>\n```\n\nCSS:\n```css\n#dropdown-menu {\n  position: absolute;\n  background-color: #f9f9f9;\n  min-width: 160px;\n  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);\n  z-index: 1;\n}","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":43,

In [15]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    index=0
    global df
    for i in range(1):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            futures = [executor.submit(inference, srno, payload) for srno in range(100)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f3c65729fed346f39fc4674a9e705cac","object":"text_completion","created":520352,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 8.118660 seconds 


Prompt 4==> Problem: Write a function in python to check if two 

Prompt 25==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-342c920d2e6f4784ba643e441a2eb74a","object":"text_completion","created":520353,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Prompt 29==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicat

Execution time: 7.728093 seconds 


Execution time: 7.605895 seconds 


Execution time: 8.052507 seconds 


Execution time: 7.545251 seconds 

Execution time: 7.825602 seconds 



Execution time: 8.250600 seconds 


Execution time: 7.640745 seconds 


Execution time: 8.151742 seconds 


Execution time: 7.583045 seconds 

Execution time: 7.704802 seconds 



Prompt 30==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-4f203ab7fd1a4bb0ac2a8424fc7c3486","object":"text_completion","created":520360,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the func


Execution time: 7.941365 seconds 

Execution time: 8.571286 seconds 

Execution time: 8.291329 seconds 



Execution time: 8.533391 seconds 


Execution time: 8.407189 seconds 




Execution time: 8.274467 seconds 


Execution time: 8.718619 seconds 


Execution time: 8.113958 seconds 


Execution time: 8.330332 seconds 


Execution time: 8.357188 seconds 

Execution time: 8.330998 seconds 



Prompt 60==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-35bde0b5907648419f139e4988d9a300","object":"text_completion","created":520367,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorte

Execution time: 8.450169 seconds 


Execution time: 8.299783 seconds 


Execution time: 8.828971 seconds 


Prompt 90==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-63c8d8e23f164b1299539102fab91171","object":"text_completion","created":520375,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 6.317636 se